In [15]:
import math
import numpy as np

# Project 1: **Molecular Geometry Analysis**

This project demonstrates the calculation of the internal coordinates (bond lengths, bond angles, dihedral angles), moments of inertia, and rotational constants of a polyatomic molecule.

### Step 1: **Read the Coordinate Data from Input**

The input to the program is the set of Cartesian coordinates of the atoms (in bohr) and their associated atomic numbers. The first line in the file is the number of atoms (an integer), while the remaining lines contain the Z-values and $x$, $y$, and $z$ coordinates of each atom (one integer followed by three double-precision floating-point numbers).
$$$$
We store this, excluding the first line, in the following format $$\begin{matrix} Z_1 & x_1 & y_1 & z_1 \\ Z_2 & x_2 & y_2 & z_2 \\ Z_3 & x_3 & y_3 & z_3 \\ \vdots & \vdots & \vdots & \vdots \end{matrix}$$

In [16]:
geom1 = np.genfromtxt("acetaldehyde.dat.txt", skip_header=1) # get the values from the data file (skip the 1st line) and store in a 2D array

### Step 2: **Bond Lengths**

Calculate the interatomic distances using the expression: $$R_{ij}=\sqrt{(x_i-x_j)^2+(y_i-y_j)^2+(z_i-z_j)^2}$$ where x, y, and z are Cartesian coordinates and i and j denote atomic indices.

In [17]:
def R(geom,i,j): # bond-length between points i-j
  return math.sqrt((geom[i,1]-geom[j,1])**2+(geom[i,2]-geom[j,2])**2+(geom[i,3]-geom[j,3])**2)


print("\n Bond lengths (bohr) \n")
for i in range(len(geom1)):
  for j in range(len(geom1)):
    if j<i and R(geom1,i,j)<3:
      print(i, j, "{:9.5f}".format(R(geom1,i,j)) )
print("\n",end="")


 Bond lengths (bohr) 

1 0   2.84511
2 1   2.29803
3 1   2.09811
4 0   2.06517
5 0   2.07407
6 0   2.07407



### Step 3: **Bond Angles**

Calculate all possible bond angles. For example, the angle, $\hspace{4pt} \phi_{ijk} \hspace{4pt}$, between atoms $\hspace{6pt} i-j-k \hspace{6pt}$, where $\hspace{4pt} j \hspace{4pt}$ is the central atom is given by $$\cos\phi_{ijk}=\hat{\mathbf{e}}_{ji}\cdot\hat{\mathbf{e}}_{jk}$$ where the $\hat{\mathbf{e}}_{ij}$ are unit vectors between the atoms, e.g. $$e_{ij}^x=-\frac{x_i-x_j}{R_{ij}}\ ,\quad e_{ij}^y=-\frac{y_i-y_j}{R_{ij}}\ ,\quad e_{ij}^z=-\frac{z_i-z_j}{R_{ij}}$$

In [18]:
def ex(geom,i,j): # the x-component of the unit vector between points i-j
  return -(geom[i,1]-geom[j,1])/R(geom,i,j)
def ey(geom,i,j): # the y-component of the unit vector between points i-j
  return -(geom[i,2]-geom[j,2])/R(geom,i,j)
def ez(geom,i,j): # the z-component of the unit vector between points i-j
  return -(geom[i,3]-geom[j,3])/R(geom,i,j)
def e(geom,i,j): # the unit vector between points i-j
  return np.array([ ex(geom,i,j), ey(geom,i,j), ez(geom,i,j) ])
def dot(v1, v2): # dot product between two vectors v1, v2 (can also use np.dot)
  return v1[0]*v2[0]+v1[1]*v2[1]+v1[2]*v2[2]
def cosphi(geom,i,j,k): # the cosine of bond-angle between points i-j-k
  return dot(e(geom,j,i), e(geom,j,k))
def phi(geom,i,j,k): # the bond-angle between points i-j-k
  return math.acos(cosphi(geom,i,j,k))

print("\n Bond angles \n")
for i in range(len(geom1)):
  for j in range(len(geom1)):
    for k in range(len(geom1)):
      if j<i and k<j and R(geom1,i,j)<4 and R(geom1,j,k)<4:
        print(i, j, k, "{:12.6f}".format(math.degrees(phi(geom1,i,j,k))) )
print("\n",end="")


 Bond angles 

2 1 0   124.268308
3 1 0   115.479341
3 2 1    28.377448
5 4 0    35.109529
6 4 0    35.109529
6 5 0    36.373677
6 5 4    60.484476



### Step 4: **Out-of-Plane Angles**

Calculate all possible out-of-plane angles. For example, the angle $\hspace{4pt} θ_{ijkl} \hspace{4pt}$ for atom $\hspace{4pt} i \hspace{4pt}$ out of the plane containing atoms $\hspace{6pt} j-k-l \hspace{6pt}$ (with $\hspace{4pt} k \hspace{4pt}$ as the central atom, connected to $\hspace{4pt} i \hspace{4pt}$) is given by $$\sin\theta_{ijkl}=\frac{\hat{\mathbf{e}}_{kj}\times\hat{\mathbf{e}}_{kl}}{\sin\phi_{jkl}}\cdot\hat{\mathbf{e}}_{ki}$$

In [19]:
def cross(v1, v2): # cross product between two vectors v1, v2 (can also use np.cross)
  return np.array([ (v1[1]*v2[2]-v1[2]*v2[1]), (v1[2]*v2[0]-v1[0]*v2[2]), (v1[0]*v2[1]-v1[1]*v2[0]) ])
def sintheta(geom,i,j,k,l): # the sine of out-of-plane angle between point i and plane containing points j-k-l(k as central point)
  return dot(cross(e(geom,k,j), e(geom,k,l)), e(geom,k,i))/math.sin(phi(geom,j,k,l))
def theta(geom,i,j,k,l): #  the out-of-plane angle between point i and plane containing points j-k-l(k as central point)
  if sintheta(geom,i,j,k,l) < -1:
    return math.asin(-1)
  elif sintheta(geom,i,j,k,l) > 1:
    return math.asin(1)
  else:
    return math.asin(sintheta(geom,i,j,k,l))


print("\n Out-of-plane angles \n")
for i in range(len(geom1)):
  for j in range(len(geom1)):
    for k in range(len(geom1)):
      for l in range(len(geom1)):
        if i!=j and i!=k and i!=l and j!=k and l<j and k!=l and R(geom1,i,k)<4 and R(geom1,k,j)<4 and R(geom1,k,l)<4:
          print(i, j, k, l, "{:12.6f}".format(math.degrees(theta(geom1,i,j,k,l))) )
print("\n",end="")


 Out-of-plane angles 

0 3 1 2    -0.000000
0 5 6 4    19.850523
0 6 4 5    19.939726
0 6 5 4   -19.850523
1 5 0 4    53.678778
1 6 0 4   -53.678778
1 6 0 5    54.977064
2 3 1 0     0.000000
3 2 1 0    -0.000000
4 5 0 1   -53.651534
4 5 6 0   -29.885677
4 6 0 1    53.651534
4 6 0 5   -54.869992
4 6 5 0    29.885677
5 4 0 1    53.626323
5 4 6 0    31.064344
5 6 0 1   -56.277112
5 6 0 4    56.194621
5 6 4 0   -30.558964
6 4 0 1   -53.626323
6 4 5 0   -31.064344
6 5 0 1    56.277112
6 5 0 4   -56.194621
6 5 4 0    30.558964



### Step 5: **Torsion/Dihedral Angles**

Calculate all possible torsional angles. For example, the torsional angle $\hspace{4pt} \tau_{ijkl} \hspace{4pt}$ for the atom connectivity $\hspace{6pt} i-j-k-l \hspace{6pt}$ is given by $$\cos\tau_{ijkl}=\frac{(\hat{\mathbf{e}}_{ij}\times\hat{\mathbf{e}}_{jk})\cdot(\hat{\mathbf{e}}_{jk}\times\hat{\mathbf{e}}_{kl})}{\sin\phi_{ijk}\sin\phi_{jkl}}$$


In [20]:
def costau(geom,i,j,k,l):
  return dot(cross(e(geom,i,j),e(geom,j,k)),cross(e(geom,j,k),e(geom,k,l)))/(math.sin(phi(geom,i,j,k))*math.sin(phi(geom,j,k,l)))
def tau(geom,i,j,k,l):
  if costau(geom,i,j,k,l)<-1:
    return math.acos(-1)
  elif costau(geom,i,j,k,l)>1:
    return math.acos(1)
  else:
    return math.acos(costau(geom,i,j,k,l))


print("\n Torsional angles \n")
for i in range(len(geom1)):
  for j in range(len(geom1)):
    for k in range(len(geom1)):
      for l in range(len(geom1)):
        if j<i and k<j and l<k and R(geom1,i,j)<4 and R(geom1,j,k)<4 and R(geom1,k,l)<4:
          print(i, j, k, l, "{:12.6f}".format(math.degrees(tau(geom1,i,j,k,l))) )
print("\n",end="")


 Torsional angles 

3 2 1 0   180.000000
6 5 4 0    36.366799



### Step 6: **Center-of-Mass**

Find the center of mass of the molecule $$x_{cm}=\frac{\sum_im_ix_i}{\sum_im_i}\ ,\quad y_{cm}=\frac{\sum_im_iy_i}{\sum_im_i}\ ,\quad z_{cm}=\frac{\sum_im_iz_i}{\sum_im_i}$$
where $m_i$ is the mass of atom $i$ and the summation runs over all atoms in the molecule
$$$$
We should, first, add another columnn for $m$ values, such that $$\begin{matrix} Z_1 & x_1 & y_1 & z_1 & m_1 \\ Z_2 & x_2 & y_2 & z_2 & m_2 \\ Z_3 & x_3 & y_3 & z_3 & m_3 \\ \vdots & \vdots & \vdots & \vdots & \vdots \end{matrix}$$

In [21]:
masses = np.array([0.,1.00782503207,4.00260325415,7.016004548,9.012182201,11.009305406, 12,14.00307400478,15.99491461956,18.998403224,19.99244017542,
22.98976928087,23.985041699,26.981538627,27.97692653246,30.973761629,31.972070999,34.968852682,39.96238312251,38.963706679,39.962590983,44.955911909,
47.947946281,50.943959507,51.940507472,54.938045141,55.934937475,58.933195048,57.935342907,62.929597474,63.929142222,68.925573587,73.921177767,
74.921596478,79.916521271,78.918337087,85.910610729,84.911789737,87.905612124,88.905848295,89.904704416,92.906378058,97.905408169,98.906254747,
101.904349312,102.905504292,105.903485715,106.90509682,113.90335854,114.903878484,119.902194676,120.903815686,129.906224399,126.904472681,131.904153457,
132.905451932,137.905247237,138.906353267,139.905438706,140.907652769,141.907723297,144.912749023,151.919732425,152.921230339,157.924103912,158.925346757,
163.929174751,164.93032207,165.930293061,168.93421325,173.938862089,174.940771819,179.946549953,180.947995763,183.950931188,186.955753109,191.96148069,
192.96292643,194.964791134,196.966568662,201.970643011,204.974427541,207.976652071,208.980398734,208.982430435,210.987496271,222.017577738,222.01755173,
228.031070292,227.027752127,232.038055325,231.03588399,238.050788247,237.048173444,242.058742611,243.06138108,247.07035354,247.07030708,251.079586788,
252.082978512,257.095104724,258.098431319,255.093241131,260.105504,263.112547,255.107398,259.114500,262.122892,263.128558,265.136151,281.162061,272.153615,
283.171792,283.176451,285.183698,287.191186,292.199786,291.206564,293.214670]) #data taken from https://github.com/psi4/psi3/blob/master/include/masses.h

def add_mass_column(geom): # to dd a column to the geom, for the mass of the atoms
  geom = np.c_[geom, np.zeros(len(geom))]  # add an all-zero column to our geom array
  for i in range(len(geom)): 
    geom[i,4] = masses[int(geom[i,0])]
  return geom

geom1=add_mass_column(geom1) # now the geom array has a 4th column, for the mass of the atoms


def center_of_mass(geom):
  cm = np.zeros([3])
  xcm_num=0
  ycm_num=0
  zcm_num=0
  denom=0
  for i in range(len(geom)):
    xcm_num += geom[i,4]*geom[i,1]
    ycm_num += geom[i,4]*geom[i,2]
    zcm_num += geom[i,4]*geom[i,3]
    denom += geom[i,4]
  cm[0] = xcm_num/denom
  cm[1] = ycm_num/denom
  cm[2] = zcm_num/denom
  return cm

cm = center_of_mass(geom1)


print("\n Molecular center of mass  {:12.8f} {:12.8f} {:12.8f} \n".format(cm[0],cm[1],cm[2]) )


 Molecular center of mass    0.64494925   0.00000000   2.31663790 



### Step 7: **Moments of Inertia**

Calculate elements of the moment of inertia tensor. $$I_{xx}=\sum_im_i\Bigl((y_i-y_{cm})^2+(z_i-z_{cm})^2\Bigr)\ ,\quad I_{yy}=\sum_im_i\Bigl((x_i-x_{cm})^2+(z_i-z_{cm})^2\Bigr)\ ,\quad I_{zz}=\sum_im_i\Bigl((x_i-x_{cm})^2+(y_i-y_{cm})^2\Bigr)\ ,\\ I_{xy}=I_{yx}=-\sum_im_i(x_i-x_{cm})(y_i-y_{cm})\ ,\quad I_{xz}=I_{xz}=-\sum_im_i(x_i-x_{cm})(z_i-z_{cm})\ ,\quad I_{yz}=I_{yz}=-\sum_im_i(y_i-y_{cm})(z_i-z_{cm})$$

In [22]:
I = np.zeros([3,3])
for i in range(len(geom1)):
  I[0,0] += geom1[i,4]*((geom1[i,2]-cm[1])**2+(geom1[i,3]-cm[2])**2)             
  I[1,1] += geom1[i,4]*((geom1[i,1]-cm[0])**2+(geom1[i,3]-cm[2])**2)
  I[2,2] += geom1[i,4]*((geom1[i,1]-cm[0])**2+(geom1[i,2]-cm[2])**2)
  I[0,1] += -geom1[i,4]*(geom1[i,1]-cm[0])*(geom1[i,2]-cm[1])
  I[0,2] += -geom1[i,4]*(geom1[i,1]-cm[0])*(geom1[i,3]-cm[2])
  I[1,2] += -geom1[i,4]*(geom1[i,2]-cm[1])*(geom1[i,3]-cm[2])
I[1,0] = I[0,1]
I[2,0] = I[0,2]
I[2,1] = I[1,2]

print("\n Moment of inertia tensor (amu\u2219bohr\u00b2) \n")
for row in I: # to print 2D numpy arrays without brackets
  print(' '.join(map(lambda I: "{:20.12f}".format(I), row)))
print("\n",end="")


 Moment of inertia tensor (amu∙bohr²) 

    156.154091420701       0.000000000000     -52.855583329239
      0.000000000000     199.371126513842       0.000000000000
    -52.855583329239       0.000000000000     290.739929664163



### Step 8: **Principal Moments of Inertia**

Calculate the eigenvalues of the inertia tensor to obtain the principal moments of inertia $\hspace{3pt} I_a \hspace{1pt}, \hspace{3pt} I_b \hspace{1pt}, \hspace{3pt} I_c$ 
$$$$
The principal axes are ordered such that associated inertia moments decrease, that is, $$I_a \leq I_b \leq I_c$$ Some authors, however, define the $a$ axis as the molecular rotation axis of highest order.

In [23]:
I_eigval, I_eigvec = np.linalg.eig(I)
I_eigval.sort() # sort the eigenvalues in ascending order
Ia = I_eigval[0]
Ib = I_eigval[1]
Ic = I_eigval[2]

print("\n Principal moments of inertia (amu bohr\u00b2) \n {:12.6f} {:12.6f} {:12.6f} \n".format(Ia,Ib,Ic))
const=0.529177249*0.529177249
print("\n Principal moments of inertia (amu \u212b\u00b2) \n {:12.6f} {:12.6f} {:12.6f} \n".format(Ia*const,Ib*const,Ic*const))
const=1.6605402e-24*0.529177249e-8*0.529177249e-8
print("\n Principal moments of inertia (g cm\u00b2) \n {:14.6e} {:14.6e} {:14.6e} \n".format(Ia*const,Ib*const,Ic*const))


 Principal moments of inertia (amu bohr²) 
   137.878035   199.371127   309.015987 


 Principal moments of inertia (amu Å²) 
    38.609788    55.829610    86.533302 


 Principal moments of inertia (g cm²) 
   6.411310e-39   9.270731e-39   1.436920e-38 



### Step 9: **Molecular Type**

Based on the relative values of the principal moments, determine the molecular rotor type
* diatomic
* $ I_a \ll I_b = I_c, \hspace{6pt} I_a \approx 0 \implies $ linear polyatomic
* $ I_a = I_b < I_c \implies $ oblate symmetric top
* $ I_a < I_b = I_c \implies $ prolate symmetric top
* $ I_a = I_b = I_c \implies $ spherical top
* $ I_a < I_b < I_c \implies $ asymmetric top




In [24]:
if len(geom1)==2:
  print("\n Molecule is diatomic. \n")
elif abs(Ia-Ib)>1e-4 and abs(Ib-Ic)<1e-4 and Ia<1e-4:
  print("\n Molecule is a linear polyatomic. \n")
elif abs(Ia-Ib)<1e-4 and abs(Ib-Ic)>1e-4:
  print("\n Molecule is an oblate symmetric top. \n")
elif abs(Ia-Ib)>1e-4 and abs(Ib-Ic)<1e-4:
  print("\n Molecule is a prolate symmetric top. \n")
elif abs(Ia-Ib)<1e-4 and abs(Ib-Ic)<1e-4:
  print("\n Molecule is a spherical top. \n")
elif abs(Ia-Ib)>1e-4 and abs(Ib-Ic)>1e-4:
  print("\n Molecule is an asymmetric top. \n")


 Molecule is an asymmetric top. 



### Step 10: **Rotational Constants**

$$A \geq B \geq C$$
$$$$
$$A=\frac{h}{8\pi^2cI_a}\ , \quad B=\frac{h}{8\pi^2cI_b}\ , \quad C=\frac{h}{8\pi^2cI_c}$$

In [25]:
h=6.6260755e-34 # in m2.kg.s-1
const=h/(8*math.pi*math.pi*1.6605402E-27*(5.29177249e-11**2))*1e-6 
print("\n Rotational constants (MHz) \n    A = {:9.3f}   B = {:9.3f}   C = {:9.3f} \n".format(const/Ia, const/Ib, const/Ic) )
c=2.99792458e10 # in cm.s-1
const=h/(8*math.pi*math.pi*c*1.6605402E-27*(5.29177249e-11**2))
print("\n Rotational constants (cm\u207b\u00b9) \n    A = {:6.4f}   B = {:6.4f}   C = {:6.4f} \n".format(const/Ia, const/Ib, const/Ic) )


 Rotational constants (MHz) 
    A = 13089.403   B =  9052.169   C =  5840.284 


 Rotational constants (cm⁻¹) 
    A = 0.4366   B = 0.3019   C = 0.1948 

